In [1]:
import numpy as np
import pandas as pd
import warnings
import sys

In [2]:
# Set float format for display (e.g., 2 decimal places)
pd.options.display.float_format = '{:.2f}'.format

# Suppress only the specific FutureWarning from pandas
warnings.filterwarnings("ignore")

In [3]:
#using DataCleaning Module

sys.path.append('/kaggle/input/modules/pyfiles')
from datacleaning import DataCleaning

In [4]:
data = pd.read_csv("/kaggle/input/london-house-price-prediction-advanced-techniques/train.csv")

# Use the class
cleaner = DataCleaning(data)
cleaner.show_info()
cleaner.drop_duplicates()
cleaner.fill_missing(strategy='median', columns=['bathrooms', 'bedrooms', 'livingRooms', 'floorAreaSqM'])
cleaner.fill_missing(strategy='mode', columns=['tenure', 'propertyType', 'currentEnergyRating'])
#cleaner.remove_outliers(columns=['price','floorAreaSqM', 'bathrooms', 'bedrooms', 'livingRooms') 
cleaner.drop_duplicates()
cleaner.show_info()

# Get the cleaned data
df = cleaner.get_clean_data()
df["price"] = df["price"]*(10**-6)   
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.describe()

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266325 entries, 0 to 266324
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   266325 non-null  int64  
 1   fullAddress          266325 non-null  object 
 2   postcode             266325 non-null  object 
 3   country              266325 non-null  object 
 4   outcode              266325 non-null  object 
 5   latitude             266325 non-null  float64
 6   longitude            266325 non-null  float64
 7   bathrooms            217846 non-null  float64
 8   bedrooms             241482 non-null  float64
 9   floorAreaSqM         252519 non-null  float64
 10  livingRooms          229285 non-null  float64
 11  tenure               260604 non-null  object 
 12  propertyType         265817 non-null  object 
 13  currentEnergyRating  209511 non-null  object 
 14  sale_month           266325 non-null  int64  
 15  sale_y

,ID,latitude,longitude,bathrooms,bedrooms,floorAreaSqM,livingRooms,sale_month,sale_year,price
count,266325.00,266325.00,266325.00,266325.00,266325.00,266325.00,266325.00,266325.00,266325.00,266325.00
mean,133162.00,51.51,-0.10,1.36,2.45,98.87,1.25,6.81,2012.21,0.62
std,76881.55,0.06,0.09,0.67,1.13,55.07,0.55,3.36,9.14,1.27
min,0.00,51.39,-0.35,1.00,1.00,10.00,1.00,1.00,1995.00,0.01
25%,66581.00,51.47,-0.16,1.00,2.00,64.00,1.00,4.00,2004.00,0.22
50%,133162.00,51.51,-0.11,1.00,2.00,85.00,1.00,7.00,2014.00,0.39
75%,199743.00,51.55,-0.05,2.00,3.00,115.00,1.00,10.00,2021.00,0.65
max,266324.00,51.67,0.14,9.00,9.00,500.00,9.00,12.00,2023.00,100.00
